In [ ]:
Analysis of PONG reinforcement learning algorithm


The idea is to find a hyperparameters that train the model to be competitive with the AI as quickly as possiable.
The hope is to be able to find a model that can be used as part of a workshop.  Ideally I'd like the training to take only as long as the workshop, so we can start it at the beginning, and check it at the end.  (which would be about 2 hours)

I define 'competitive with the AI' as obtaining a mean rewards >= 0.0 over 100 episodes.

Depending on the analysis, it might be possiable to train the model faster using parallism.  I'd be happy to rent up to 32 cores, so we should be able to get enough CPU time.  Might have to shift to TF for this, and might have to parallise the enviroment evaluation (this could be done by launching multiple threads and running each episode in the batch independantly... I kind of like the idea of this).

Key outcomes
Find a model that can be trained to 0ev as quick as possiable (<8000 episodes) 

Key questions

does weight decay help?
what do weights look like over time


## The initial model

Training on the initial model progresses very slowly.  We 


## Improved baseline

In order to train these nets in a reaosnable time, we will use an updated baseline model with the following parameters.

H = 100
Learning_rate = 3e-3
Weight_decay = 0.01

<pic>

comment in varience in results.

Analysis of gameplay
<gif>

The algo really loves to get it into a corner.  Also sometimes it gets 'on a roll' and wills very quickly.  Other times it gets into a loosing streak, where it can't seem to get out of it.  This makes the outcomes very diometric.

Also note: looks like episodes are limited to 10000 frames.  This is a very long game, and it happens quite a bit.  Shows that the algorithm can get in a loop where we just bounce back and foward with the ai.



<histogram of scores>


# Investigating the Hyperparameters

## Hidden Units

200 Hidden units seems like a bit too much.  We are working with a screen resolution of 80x80 pixels, so somewhere around 80 would be a good place to start (this allows the model to track the other players location, as well as ...)

### Looking at the weights

### Extreme cases

Can we learn with just two hidden units?  One for up and one for down?  

>> image

It seems this does work.  I wonder at what point it would platue.  I thought it might super impose various siutions but it doesn't seem to do that.

Does H=1 work?  (This is just a single layer net)

Nope... this is not able to learn at all.  It does come up with a similar weights distribution though.

Takeaway:
Single layer has no effect, but even a very basic 2 layer model works quite well.  H=100 works best

## Learning rate

This is proabbly the most important hyper paremter.  We try a range of learning rates in half order of magnitude steps.

<pic>

The 1e-4 learning rate is far too slow.  Interstingly setting a higher learning rate (1e-2) is also very poor.  The learning rates inbetween all do very well.  it seems that 3e3 is the best for our goal of training fast, however I suspect that 1e3 would be much better if we want the algorithm to progress further.  

Takeaway
Train at 3e3, then switch to 1e3 after about 4000 iterations.



## Discount Rate

<info on discount rate>

<pic>

High discount rates do not perofrm well.  The original discount rate of 0.99 works very well.  Something that doesn't show up on this graph, however, is the varience in the scores.  I found that <low??> discount rates peformed much more constantly than higher ones.  This actaully made the higher discount rate models train much faster, as the games ended quicker.

Ideas... change discount rate, maybe 1k at .99 and 1k at 0.9?  I.e. learn the 'short game' and the 'long game'.

<pic of iterations over time> <pic of varience over time>

Takeaway
The discount rate of .99 is suitable for our needs, but if constant results are required a smaller discount rate is important.

(this introduces an interesting notion, that is lost when looking only at ev... which is what is the worst case situation, and how predicatble is the result.  A -21 +21 AI is very differetn from a 0 / 0 bot, that always hits the ball back slowly.



Also:
We could introduce a very small penaltiy as the game goes on.  This might encourage the algorithm to play faster, which would help with training times and be less frustrating to watch (stalemate is 0ev, but not very interesting to watch).  Another modification might be to change the reward structure, say +2 for a point and -1 against (i.e. play aggressively) or +1 for a point and -2 against (i.e. make sure not to give up and points).

## Batch_size

How many games before we apply changes?

<pic>

The batch size doesn't make much of a difference, except for the special case of bs = 1.  In this case the algorithm performs very poorly.  This may be because games can go one of two ways (as mentioned in the introduction) having a batch size of 2 we will even out a lot of these experiences.

Also of note, is that a batch size of 10 slightly underperforms the smaller batches, but ends up being more constant at the end.  For this reason if we where training the best net possiable we'd proabbly want to use batch size of 10.

Also I thought high batch sizes would slow things down a lot as we make updates more slowly.  But it has little effect.

takeway
The batch size of 5 is optimal for our purposes, but training for a better net 10 would be better




## Weight decay

<pic>

This made less of a difference than I thought.  Setting a small weight decay does help speed up the training at the begining, but may limit the peformance at the high end.

A look at the weights produced with no weight decay and weight decay:
<pic>
Andrea speculated that the weight decay would reduced the noise, however it make very little difference in this case.  It seems the noise is mostly introduced from the location of the ball as it moves?  But that sounds unlikely too as some corners would be less common

takeaway
A slightly smaller weight decay of 1e-3 would be better, even smaller if we wanted to continue training.

In [ ]:
## Gamma

This had very little effect.

<pic>

Values of .999 and .995 stop the learning process.  All others work ok.

Take away:
.99 gamma is fine.


## Training system

At this point I should mention how I trained these nets.  Each model took around 20 hours to train, so many of those graphs represent ~100 hours of compute time.

In order to manage the workload I created a script <github> that watches a file and starts work on any models that need training.  I sync everything up with dropbox, this way I can simple create a new model on my laptop, and a few minutes late a computer in the cloud picks it up and works on it while I sleep.

I ended up using 4 computers as part of a (very makeshift) cluster.

A dell linux dual core xeon
My personal desktop quad core (windows, so I had to run the trainer under VPC)
my dual core macbook pro 
A 4VCPU cloud computer (this was by far the fastest)  Thanks to <link> for giving me some free credit on this, it made a big difference

Totalling 12 cores

I also created a script <pic> to monitor the progress of the models.

I must admit I spent way too much time on this.

I found very little peformance gain running the models on more than one core (this might be a limitation with numpy and relatively small matrices) so I forced each model to train on a single CPU (amazingly this actaully speed up the training time - presumably because of overheards), and then trained multiple models in parallel.  For example on my 4 core desktop I trained 4 models (or 3 if I was using it) at a time.




# Final model

....


# NExt steps

Onward to tensorflow and convnets.

Looking forward to seeing if it's faster, also if GPU will help.  Enviroment might be a problem, if it is I'll try threading that up.  Also, if we are working in batches of say 8, we can run these 8 episodes in paralell, waiting only on the last one.
